In [1]:
pip install pyomo

Note: you may need to restart the kernel to use updated packages.


In [1]:
!pip install cplex

ERROR: Could not find a version that satisfies the requirement cplex (from versions: none)
ERROR: No matching distribution found for cplex


In [ ]:
import pyomo.environ as pyo

# --- 1. Veri Hazırlama ---
# Kapsamak istediğimiz rotaların listesi
rotalar = ["i1_Ank_Izm", "i2_Ank_Gebze", "i3_Bolu_Izm", "i4_Gerede_Gebze"]

# Sensör kurabileceğimiz potansiyel konumların listesi
konumlar = ["j1_Polatli", "j2_Gerede", "j3_Bolu", "j4_Susurluk", "j5_Gebze"]

# Her bir konuma sensör kurmanın maliyeti (en az sayıyı bulmak için hepsi 1 alındı)
maliyetler = {
    "j1_Polatli": 1,
    "j2_Gerede": 1,
    "j3_Bolu": 1,
    "j4_Susurluk": 1,
    "j5_Gebze": 1,
}

# Kapsama Matrisi (a_ij): Hangi konum (j) hangi rotayı (i) kapsıyor?
# (rota, konum) -> 1 (kapsıyor) veya 0 (kapsamıyor)
kapsama_matrisi = {}
for i in rotalar:
    for j in konumlar:
        kapsama_matrisi[i, j] = 0

# Şimdi sadece kapsayan (1 olan) durumları güncelleyelim
# i1 (Ank -> İz) rotasını j1 (Polatlı) ve j4 (Susurluk) görüyor
kapsama_matrisi["i1_Ank_Izm", "j1_Polatli"] = 1
kapsama_matrisi["i1_Ank_Izm", "j4_Susurluk"] = 1

# i2 (Ank -> Gebze) rotasını j1, j2, j3, j5 görüyor
kapsama_matrisi["i2_Ank_Gebze", "j1_Polatli"] = 1
kapsama_matrisi["i2_Ank_Gebze", "j2_Gerede"] = 1
kapsama_matrisi["i2_Ank_Gebze", "j3_Bolu"] = 1
kapsama_matrisi["i2_Ank_Gebze", "j5_Gebze"] = 1

# i3 (Bolu -> İz) rotasını j3 (Bolu) ve j4 (Susurluk) görüyor
kapsama_matrisi["i3_Bolu_Izm", "j3_Bolu"] = 1
kapsama_matrisi["i3_Bolu_Izm", "j4_Susurluk"] = 1

# i4 (Gerede -> Gebze) rotasını j2 (Gerede), j3 (Bolu) ve j5 (Gebze) görüyor
kapsama_matrisi["i4_Gerede_Gebze", "j2_Gerede"] = 1
kapsama_matrisi["i4_Gerede_Gebze", "j3_Bolu"] = 1
kapsama_matrisi["i4_Gerede_Gebze", "j5_Gebze"] = 1


# --- 2. Pyomo Modelini Kurma ---
model = pyo.ConcreteModel(name="KGM_Sensor_Yerlesimi")

# --- 3. Kümeleri (Sets) Tanımlama ---
model.I_Rotalar = pyo.Set(initialize=rotalar)
model.J_Konumlar = pyo.Set(initialize=konumlar)

# --- 4. Parametreleri (Parameters) Tanımlama ---
model.c_Maliyet = pyo.Param(model.J_Konumlar, initialize=maliyetler)
model.a_Kapsama = pyo.Param(
    model.I_Rotalar, model.J_Konumlar, initialize=kapsama_matrisi
)

# --- 5. Karar Değişkenini (Decision Variable) Tanımlama ---
model.x_KurulumKarari = pyo.Var(model.J_Konumlar, domain=pyo.Binary)


# --- 6. Amaç Fonksiyonunu (Objective Function) Tanımlama ---
def amac_kurali(m):
    return sum(m.c_Maliyet[j] * m.x_KurulumKarari[j] for j in m.J_Konumlar)


model.Amac = pyo.Objective(rule=amac_kurali, sense=pyo.minimize)


# --- 7. Kısıtları (Constraints) Tanımlama ---
def kapsama_kurali(m, i):
    return sum(m.a_Kapsama[i, j] * m.x_KurulumKarari[j] for j in m.J_Konumlar) >= 1


model.KapsamaKisitlari = pyo.Constraint(model.I_Rotalar, rule=kapsama_kurali)

# --- 8. Modeli Çözme ve Sonuçları Gösterme ---
glpk_yolu = r"C:\glpk\glpk-4.65\w64\glpsol.exe"
solver = pyo.SolverFactory("glpk", executable=glpk_yolu)
sonuc = solver.solve(model)

# --- 9. Sonuçları Ekrana Yazdırma ---
print("--- KGM SENSÖR YERLEŞİMİ OPTİMAL ÇÖZÜMÜ ---")
print(f"Model Durumu: {sonuc.solver.termination_condition}")
print("\n--- KURULMASI GEREKEN SENSÖRLER ---")

toplam_maliyet = 0
kurulan_sensorler = []

for j in model.J_Konumlar:
    if pyo.value(model.x_KurulumKarari[j]) > 0.5:
        print(f"  [+] Konum: {j}")
        kurulan_sensorler.append(j)
        toplam_maliyet += pyo.value(model.c_Maliyet[j])

print("\n--- ÖZET ---")
print(f"Toplam Kurulacak Sensör Sayısı: {len(kurulan_sensorler)}")
print(f"Toplam Minimum Maliyet: {toplam_maliyet} (birim)")

print("\n--- KAPSAMA KONTROLÜ ---")
for i in rotalar:
    kapsayan_sensorler = []
    for j in kurulan_sensorler:
        if kapsama_matrisi[i, j] == 1:
            kapsayan_sensorler.append(j)
    print(f"  -> {i} rotası, {kapsayan_sensorler} tarafından kapsandı. (OK)")

--- KGM SENSÖR YERLEŞİMİ OPTİMAL ÇÖZÜMÜ ---
Model Durumu: optimal

--- KURULMASI GEREKEN SENSÖRLER ---
  [+] Konum: j1_Polatli
  [+] Konum: j3_Bolu

--- ÖZET ---
Toplam Kurulacak Sensör Sayısı: 2
Toplam Minimum Maliyet: 2 (birim)

--- KAPSAMA KONTROLÜ ---
  -> i1_Ank_Izm rotası, ['j1_Polatli'] tarafından kapsandı. (OK)
  -> i2_Ank_Gebze rotası, ['j1_Polatli', 'j3_Bolu'] tarafından kapsandı. (OK)
  -> i3_Bolu_Izm rotası, ['j3_Bolu'] tarafından kapsandı. (OK)
  -> i4_Gerede_Gebze rotası, ['j3_Bolu'] tarafından kapsandı. (OK)


In [2]:
import pyomo.environ as pyo

# --- 1. Veri Hazırlama (Yeni, Daha Gerçekçi Senaryo: Uşak Bypass) ---

# Kapsamak istediğimiz rotaların listesi (I Kümesi)
rotalar = [
    "i1_Durust_Gecis",  # Uşak'taki ana KDİ'den dürüstçe geçen tırlar
    "i2_Kuzey_Kacis_Yolu",  # Uşak'ı kuzeydeki köy yollarından bypass edenler
    "i3_Guney_Kacis_Yolu",  # Uşak'ı güneydeki ilçe yollarından bypass edenler
]

# Sensör kurabileceğimiz potansiyel konumların listesi (J Kümesi)
konumlar = [
    "j1_Usak_Dogu_Bogazi",  # KAÇIŞ ÖNCESİ: Tüm trafiğin girdiği ana boğaz (D-300)
    "j2_Kuzey_Kacis_Yolu",  # KAÇIŞ ROTASI 1: Kuzey kaçış yolu üzeri
    "j3_Guney_Kacis_Yolu",  # KAÇIŞ ROTASI 2: Güney kaçış yolu üzeri
    "j4_Merkez_KDI_Guncel",  # DÜRÜST ROTA: Mevcut KDİ'ye sensör takma
    "j5_Usak_Bati_Bogazi",  # KAÇIŞ SONRASI: Tüm trafiğin toplandığı ana boğaz (D-300)
]

# Her bir konuma sensör kurmanın maliyeti (Birim: Milyon TL)
# Not: Maliyetleri notlarına ve varsayımlara dayandırdık.
maliyetler = {
    "j1_Usak_Dogu_Bogazi": 12,  # Ana boğaz, kurulumu karmaşık ve pahalı
    "j2_Kuzey_Kacis_Yolu": 8,  # İlçe yolu, daha ucuz
    "j3_Guney_Kacis_Yolu": 8,  # İlçe yolu, daha ucuz
    "j4_Merkez_KDI_Guncel": 5,  # Sadece sensör ekleme, en ucuz maliyet
    "j5_Usak_Bati_Bogazi": 10,  # Standart ana yol kurulum maliyeti
}

# Kapsama Matrisi (a_ij): Hangi konum (j) hangi rotayı (i) kapsıyor?
# (rota, konum) -> 1 (kapsıyor) veya 0 (kapsamıyor)
kapsama_matrisi = {}
for i in rotalar:
    for j in konumlar:
        kapsama_matrisi[i, j] = 0

# Verileri dolduralım:
# i1 (Dürüst Rota): j1 (Doğu Boğazı), j4 (Merkez) ve j5 (Batı Boğazı)'ndan geçer
kapsama_matrisi["i1_Durust_Gecis", "j1_Usak_Dogu_Bogazi"] = 1
kapsama_matrisi["i1_Durust_Gecis", "j4_Merkez_KDI_Guncel"] = 1
kapsama_matrisi["i1_Durust_Gecis", "j5_Usak_Bati_Bogazi"] = 1

# i2 (Kuzey Kaçış): j1 (Doğu Boğazı), j2 (Kuzey Yolu) ve j5 (Batı Boğazı)'ndan geçer
# NOT: j4'ten (Merkez) geçmez!
kapsama_matrisi["i2_Kuzey_Kacis_Yolu", "j1_Usak_Dogu_Bogazi"] = 1
kapsama_matrisi["i2_Kuzey_Kacis_Yolu", "j2_Kuzey_Kacis_Yolu"] = 1
kapsama_matrisi["i2_Kuzey_Kacis_Yolu", "j5_Usak_Bati_Bogazi"] = 1

# i3 (Güney Kaçış): j1 (Doğu Boğazı), j3 (Güney Yolu) ve j5 (Batı Boğazı)'ndan geçer
# NOT: j4'ten (Merkez) geçmez!
kapsama_matrisi["i3_Guney_Kacis_Yolu", "j1_Usak_Dogu_Bogazi"] = 1
kapsama_matrisi["i3_Guney_Kacis_Yolu", "j3_Guney_Kacis_Yolu"] = 1
kapsama_matrisi["i3_Guney_Kacis_Yolu", "j5_Usak_Bati_Bogazi"] = 1


# --- 2. Pyomo Modelini Kurma ---
model = pyo.ConcreteModel(name="KGM_Sensor_Yerlesimi_Usak")

# --- 3. Kümeleri (Sets) Tanımlama ---
model.I_Rotalar = pyo.Set(initialize=rotalar)
model.J_Konumlar = pyo.Set(initialize=konumlar)

# --- 4. Parametreleri (Parameters) Tanımlama ---
model.c_Maliyet = pyo.Param(model.J_Konumlar, initialize=maliyetler)
model.a_Kapsama = pyo.Param(
    model.I_Rotalar, model.J_Konumlar, initialize=kapsama_matrisi
)

# --- 5. Karar Değişkenini (Decision Variable) Tanımlama ---
model.x_KurulumKarari = pyo.Var(model.J_Konumlar, domain=pyo.Binary)


# --- 6. Amaç Fonksiyonunu (Objective Function) Tanımlama ---
def amac_kurali(m):
    return sum(m.c_Maliyet[j] * m.x_KurulumKarari[j] for j in m.J_Konumlar)


model.Amac = pyo.Objective(rule=amac_kurali, sense=pyo.minimize)


# --- 7. Kısıtları (Constraints) Tanımlama ---
def kapsama_kurali(m, i):
    # KURAL 2: Her 'i' rotası için, o rotayı kapsayan j'lerin
    # (a_ij * x_j) toplamı en az 1 olmalı.
    return sum(m.a_Kapsama[i, j] * m.x_KurulumKarari[j] for j in m.J_Konumlar) >= 1


model.KapsamaKisitlari = pyo.Constraint(model.I_Rotalar, rule=kapsama_kurali)

# --- 8. Modeli Çözme ve Sonuçları Gösterme ---
# GLPK yolunu kendi bilgisayarına göre ayarla
glpk_yolu = r"C:\glpk\glpk-4.65\w64\glpsol.exe"  # ÖNEMLİ: Bu yolu kendine göre düzelt
solver = pyo.SolverFactory("glpk", executable=glpk_yolu)
sonuc = solver.solve(model)

# --- 9. Sonuçları Ekrana Yazdırma ---
print("--- KGM SENSÖR YERLEŞİMİ OPTİMAL ÇÖZÜMÜ (UŞAK SENARYOSU) ---")
print(f"Model Durumu: {sonuc.solver.termination_condition}")
print("\n--- KURULMASI GEREKEN SENSÖRLER ---")

toplam_maliyet = 0
kurulan_sensorler = []

for j in model.J_Konumlar:
    if pyo.value(model.x_KurulumKarari[j]) > 0.5:
        print(f"  [+] Konum: {j}")
        kurulan_sensorler.append(j)
        toplam_maliyet += pyo.value(model.c_Maliyet[j])

print("\n--- ÖZET ---")
print(f"Toplam Kurulacak Sensör Sayısı: {len(kurulan_sensorler)}")
print(f"Toplam Minimum Maliyet: {toplam_maliyet} Milyon TL")

print("\n--- KAPSAMA KONTROLÜ ---")
for i in rotalar:
    kapsayan_sensorler = []
    for j in kurulan_sensorler:
        if kapsama_matrisi[i, j] == 1:
            kapsayan_sensorler.append(j)
    print(f"  -> {i} rotası, {kapsayan_sensorler} tarafından kapsandı. (OK)")

--- KGM SENSÖR YERLEŞİMİ OPTİMAL ÇÖZÜMÜ (UŞAK SENARYOSU) ---
Model Durumu: optimal

--- KURULMASI GEREKEN SENSÖRLER ---
  [+] Konum: j5_Usak_Bati_Bogazi

--- ÖZET ---
Toplam Kurulacak Sensör Sayısı: 1
Toplam Minimum Maliyet: 10 Milyon TL

--- KAPSAMA KONTROLÜ ---
  -> i1_Durust_Gecis rotası, ['j5_Usak_Bati_Bogazi'] tarafından kapsandı. (OK)
  -> i2_Kuzey_Kacis_Yolu rotası, ['j5_Usak_Bati_Bogazi'] tarafından kapsandı. (OK)
  -> i3_Guney_Kacis_Yolu rotası, ['j5_Usak_Bati_Bogazi'] tarafından kapsandı. (OK)


In [1]:
import pyomo.environ as pyo

# --- 1. VERİ HAZIRLAMA: KIZILCAHAMAM-KAZAN (D-750) VAKA ANALİZİ ---
# Dayanak: KGM verileri ve Uydu Görüntüsü

# Rotalar (I Kümesi): Tırların izleyebileceği güzergahlar
rotalar = [
    "i1_Ana_Yol_Durust",  # D-750 üzerinden Pınar İstasyonu'ndan geçenler
    "i2_Eski_Yol_Kacis",  # Uydu görüntüsündeki soldaki tali yoldan kaçanlar
]

# Potansiyel Sensör Konumları (J Kümesi)
konumlar = [
    "j1_Pinar_Istasyonu",  # Mevcut Trafik Denetleme noktası (Fotoğraftaki yer)
    "j2_Eski_Yol_Uzeri",  # Kaçak yolun üzeri
    "j3_Kazan_Giris_Bogazi",  # İki yolun birleştiği, kaçışın bittiği ortak nokta
]

# Maliyetler (Milyon TL - Tahmini)
# Mantık: Mevcut istasyona sensör eklemek en ucuzudur (Altyapı hazır).
# Tali yola kurmak orta maliyetli, ana yol boğazına sıfırdan kurmak en pahalıdır.
maliyetler = {
    "j1_Pinar_Istasyonu": 5,  # En ucuz (Mevcut altyapı var)
    "j2_Eski_Yol_Uzeri": 8,  # Orta (Yol dar, kurulum kolay ama elektrik yok)
    "j3_Kazan_Giris_Bogazi": 12,  # Pahalı (Ana yol, sıfırdan kurulum)
}

# Kapsama Matrisi (a_ij): Kim nereyi görüyor?
kapsama_matrisi = {}
for i in rotalar:
    for j in konumlar:
        kapsama_matrisi[i, j] = 0

# --- Matrisin Doldurulması ---

# SENARYO 1: Dürüst Rota (i1)
# Bu rota hem mevcut istasyondan (j1) hem de Kazan girişinden (j3) geçer.
# Eski yoldan (j2) GEÇMEZ.
kapsama_matrisi["i1_Ana_Yol_Durust", "j1_Pinar_Istasyonu"] = 1
kapsama_matrisi["i1_Ana_Yol_Durust", "j3_Kazan_Giris_Bogazi"] = 1

# SENARYO 2: Kaçak Rota (i2)
# Bu rota mevcut istasyondan (j1) KAÇAR (0).
# Eski yol üzerindeki sensörden (j2) ve mecburen Kazan girişinden (j3) geçer.
kapsama_matrisi["i2_Eski_Yol_Kacis", "j2_Eski_Yol_Uzeri"] = 1
kapsama_matrisi["i2_Eski_Yol_Kacis", "j3_Kazan_Giris_Bogazi"] = 1


# --- 2. MODEL KURULUMU (Set Covering Problem) ---
model = pyo.ConcreteModel(name="KGM_D750_Analizi")

# Kümeler ve Parametreler
model.I = pyo.Set(initialize=rotalar)
model.J = pyo.Set(initialize=konumlar)
model.c = pyo.Param(model.J, initialize=maliyetler)
model.a = pyo.Param(model.I, model.J, initialize=kapsama_matrisi)

# Karar Değişkeni (x_j: 1 ise kur, 0 ise kurma)
model.x = pyo.Var(model.J, domain=pyo.Binary)


# Amaç Fonksiyonu: Toplam Maliyeti Minimize Et
def obj_rule(m):
    return sum(m.c[j] * m.x[j] for j in m.J)


model.Obj = pyo.Objective(rule=obj_rule, sense=pyo.minimize)


# Kısıt: Her rota (i) en az 1 sensör tarafından yakalanmalı
def cover_rule(m, i):
    return sum(m.a[i, j] * m.x[j] for j in m.J) >= 1


model.Constraints = pyo.Constraint(model.I, rule=cover_rule)

# --- 3. ÇÖZÜM ---
# GLPK yolunu kendi bilgisayarına göre ayarla
glpk_path = r"C:\glpk\glpk-4.65\w64\glpsol.exe"
solver = pyo.SolverFactory("glpk", executable=glpk_path)
result = solver.solve(model)

# --- 4. SONUÇLARI YAZDIRMA ---
print("\n--- D-750 KIZILCAHAMAM-KAZAN PİLOT BÖLGE SONUÇLARI ---")
print(f"Çözüm Durumu: {result.solver.termination_condition}")

toplam_maliyet = 0
secilenler = []

for j in model.J:
    if pyo.value(model.x[j]) > 0.5:
        secilenler.append(j)
        toplam_maliyet += pyo.value(model.c[j])
        print(f"✅ SEÇİLEN KONUM: {j} (Maliyet: {maliyetler[j]} Milyon TL)")

print("-" * 50)
print(f"TOPLAM MALİYET: {toplam_maliyet} Milyon TL")
print("-" * 50)

print("ANALİZ:")
if "j3_Kazan_Giris_Bogazi" in secilenler and len(secilenler) == 1:
    print("Model, 'Kazan Giriş Boğazı'nı seçti.")
    print("NEDEN? Çünkü 12 Milyon TL vererek tek bir noktada")
    print("hem dürüst araçları hem de kaçakları yakalamak,")
    print("iki ayrı noktaya (5 + 8 = 13 Milyon) sistem kurmaktan daha ucuz.")
elif "j1_Pinar_Istasyonu" in secilenler:
    print("Model, ayrı ayrı kapatmayı seçti.")
    print("NEDEN? Çünkü Kazan girişine büyük bir sistem kurmak çok pahalıydı.")
    print("Bunun yerine mevcut istasyonu kullanıp, sadece kaçak yola ek sensör koydu.")


--- D-750 KIZILCAHAMAM-KAZAN PİLOT BÖLGE SONUÇLARI ---
Çözüm Durumu: optimal
✅ SEÇİLEN KONUM: j3_Kazan_Giris_Bogazi (Maliyet: 12 Milyon TL)
--------------------------------------------------
TOPLAM MALİYET: 12 Milyon TL
--------------------------------------------------
ANALİZ:
Model, 'Kazan Giriş Boğazı'nı seçti.
NEDEN? Çünkü 12 Milyon TL vererek tek bir noktada
hem dürüst araçları hem de kaçakları yakalamak,
iki ayrı noktaya (5 + 8 = 13 Milyon) sistem kurmaktan daha ucuz.


In [2]:
import pyomo.environ as pyo

# --- 1. VERİ HAZIRLAMA: GENİŞLETİLMİŞ D-750 SENARYOSU ---

# Rotalar (I Kümesi)
rotalar = [
    "i1_Durust_Transit",  # Ana yoldan dümdüz gidenler
    "i2_Eski_Yol_Kacagi",  # İstasyon arkasındaki tali yoldan kaçanlar
    "i3_Koy_Yolu_Kacagi",  # Dağ köylerinden (uzak bypass) kaçanlar
    "i4_Tas_Ocagi_Cikisi",  # Yolun ortasından (istasyon sonrası) katılanlar
]

# Potansiyel Konumlar (J Kümesi)
konumlar = [
    "j1_Kizilcahamam_Cikis",  # Rota başı
    "j2_Pinar_Istasyon_Onu",  # Mevcut istasyon
    "j3_Eski_Yol_Kuzey",  # Kaçak yol girişi
    "j4_Eski_Yol_Guney",  # Kaçak yol çıkışı
    "j5_Koy_Yolu_Gecisi",  # Zorlu dağ yolu
    "j6_Sanayi_Kavsagi",  # Ara katılım noktası
    "j7_Kazan_Giris_Bogazi",  # Rota sonu (Huni ağzı)
]

# Maliyetler (Milyon TL - Mantıksal Varsayımlar)
# Kriterler: Mevcut yerler ucuz, dağ başı (altyapısız) yerler pahalı.
maliyetler = {
    "j1_Kizilcahamam_Cikis": 10,  # Şehir içi kurulum, orta maliyet
    "j2_Pinar_Istasyon_Onu": 4,  # Mevcut altyapı var, EN UCUZ
    "j3_Eski_Yol_Kuzey": 8,  # Elektrik çekilmesi lazım
    "j4_Eski_Yol_Guney": 8,  # Elektrik çekilmesi lazım
    "j5_Koy_Yolu_Gecisi": 15,  # Dağ başı, güneş paneli+uydu lazım (Çok Pahalı)
    "j6_Sanayi_Kavsagi": 9,  # Kavşak düzenlemesi lazım
    "j7_Kazan_Giris_Bogazi": 12,  # Geniş ana yol, büyük TAG kurulumu lazım
}

# Kapsama Matrisi (a_ij) - KİM NEREDEN GEÇİYOR?
# 1: Geçer (Yakalanır) | 0: Geçmez (Yakalanmaz)
kapsama_matrisi = {}
for i in rotalar:
    for j in konumlar:
        kapsama_matrisi[i, j] = 0

# --- Matrisin Doldurulması (Senaryo Mantığı) ---

# i1 (Dürüst): Ana yoldaki tüm noktalardan geçer.
# Geçtiği yerler: j1, j2, j6, j7. (Eski yola ve köy yoluna girmez)
kapsama_matrisi["i1_Durust_Transit", "j1_Kizilcahamam_Cikis"] = 1
kapsama_matrisi["i1_Durust_Transit", "j2_Pinar_Istasyon_Onu"] = 1
kapsama_matrisi["i1_Durust_Transit", "j6_Sanayi_Kavsagi"] = 1
kapsama_matrisi["i1_Durust_Transit", "j7_Kazan_Giris_Bogazi"] = 1

# i2 (Eski Yol Kaçağı): Ana istasyondan (j2) kaçar.
# Geçtiği yerler: j1 (giriş), j3 (sapma), j4 (bağlanma), j6, j7.
kapsama_matrisi["i2_Eski_Yol_Kacagi", "j1_Kizilcahamam_Cikis"] = 1
kapsama_matrisi["i2_Eski_Yol_Kacagi", "j3_Eski_Yol_Kuzey"] = 1
kapsama_matrisi["i2_Eski_Yol_Kacagi", "j4_Eski_Yol_Guney"] = 1
kapsama_matrisi["i2_Eski_Yol_Kacagi", "j6_Sanayi_Kavsagi"] = 1
kapsama_matrisi["i2_Eski_Yol_Kacagi", "j7_Kazan_Giris_Bogazi"] = 1

# i3 (Köy Yolu Kaçağı): İstasyonu (j2) ve Eski Yolu (j3, j4) tamamen bypass eder.
# Geçtiği yerler: j1 (belki), j5 (köy yolu), j7 (en sonda bağlanır).
# Not: Sanayi kavşağından (j6) bile geçmeyebilir, dağdan gelir.
kapsama_matrisi["i3_Koy_Yolu_Kacagi", "j1_Kizilcahamam_Cikis"] = 1
kapsama_matrisi["i3_Koy_Yolu_Kacagi", "j5_Koy_Yolu_Gecisi"] = 1
kapsama_matrisi["i3_Koy_Yolu_Kacagi", "j7_Kazan_Giris_Bogazi"] = 1

# i4 (Taş Ocağı Katılımı): Yolun ortasından girer. Başlangıcı (j1, j2, j3, j4, j5) pas geçer.
# Geçtiği yerler: j6 (katılım noktası), j7 (bitiş).
kapsama_matrisi["i4_Tas_Ocagi_Cikisi", "j6_Sanayi_Kavsagi"] = 1
kapsama_matrisi["i4_Tas_Ocagi_Cikisi", "j7_Kazan_Giris_Bogazi"] = 1


# --- 2. MODEL KURULUMU ---
model = pyo.ConcreteModel(name="KGM_Genisletilmis_Model")

model.I = pyo.Set(initialize=rotalar)
model.J = pyo.Set(initialize=konumlar)
model.c = pyo.Param(model.J, initialize=maliyetler)
model.a = pyo.Param(model.I, model.J, initialize=kapsama_matrisi)

model.x = pyo.Var(model.J, domain=pyo.Binary)


def obj_rule(m):
    return sum(m.c[j] * m.x[j] for j in m.J)


model.Obj = pyo.Objective(rule=obj_rule, sense=pyo.minimize)


def cover_rule(m, i):
    return sum(m.a[i, j] * m.x[j] for j in m.J) >= 1


model.Constraints = pyo.Constraint(model.I, rule=cover_rule)

# --- 3. ÇÖZÜM ---
# GLPK yolunu kendi bilgisayarına göre ayarla
glpk_path = r"C:\glpk\glpk-4.65\w64\glpsol.exe"
solver = pyo.SolverFactory("glpk", executable=glpk_path)
result = solver.solve(model)

# --- 4. SONUÇLARI YAZDIRMA ---
print("\n--- GENİŞLETİLMİŞ KIZILCAHAMAM-KAZAN ANALİZİ ---")
print(f"Çözüm Durumu: {result.solver.termination_condition}")

toplam_maliyet = 0
secilenler = []

print("\nSEÇİLEN SENSÖR KONUMLARI:")
for j in model.J:
    if pyo.value(model.x[j]) > 0.5:
        secilenler.append(j)
        toplam_maliyet += pyo.value(model.c[j])
        print(f"✅ {j} (Maliyet: {maliyetler[j]} Milyon TL)")

print("-" * 50)
print(f"TOPLAM YATIRIM MALİYETİ: {toplam_maliyet} Milyon TL")
print("-" * 50)

print("\nROTALARIN KAPSAMA ANALİZİ:")
for i in rotalar:
    kapsayanlar = [j for j in secilenler if kapsama_matrisi[i, j] == 1]
    durum = "KAPSANDI" if kapsayanlar else "AÇIKTA KALDI"
    print(f"🚚 {i}: {durum} -> {kapsayanlar}")


--- GENİŞLETİLMİŞ KIZILCAHAMAM-KAZAN ANALİZİ ---
Çözüm Durumu: optimal

SEÇİLEN SENSÖR KONUMLARI:
✅ j7_Kazan_Giris_Bogazi (Maliyet: 12 Milyon TL)
--------------------------------------------------
TOPLAM YATIRIM MALİYETİ: 12 Milyon TL
--------------------------------------------------

ROTALARIN KAPSAMA ANALİZİ:
🚚 i1_Durust_Transit: KAPSANDI -> ['j7_Kazan_Giris_Bogazi']
🚚 i2_Eski_Yol_Kacagi: KAPSANDI -> ['j7_Kazan_Giris_Bogazi']
🚚 i3_Koy_Yolu_Kacagi: KAPSANDI -> ['j7_Kazan_Giris_Bogazi']
🚚 i4_Tas_Ocagi_Cikisi: KAPSANDI -> ['j7_Kazan_Giris_Bogazi']


In [3]:
import pyomo.environ as pyo

# --- 1. VERİ HAZIRLAMA: ANKARA-BOLU BÖLGESEL AĞ MODELİ ---

# Rotalar (I Kümesi): Bir tırın Ankara'dan Bolu yönüne gidebileceği tüm senaryolar
rotalar = [
    "i1_Full_Durust_Otoyolcu",  # Kazan -> D750 -> O4 Otoyolu (Hiç kaçağa girmez)
    "i2_Durust_DevletYolcu",  # Kazan -> D750 -> Gerede -> D100 (Devlet yolunu kullanır)
    "i3_Pinar_Kacagi",  # İstasyonun arkasındaki tali yoldan kaçan (Senin ilk vakan)
    "i4_Kizilcahamam_Dagcisi",  # Daha Kızılcahamam'dayken dağ yoluna sapıp istasyonu uzaktan geçen
    "i5_Gerede_Bypass",  # İstasyonu geçer ama Gerede kavşağındaki denetimden kaçmak için köy yoluna girer
    "i6_Eski_Bolu_Yolu",  # Ana yolları tamamen bırakıp çok eski, bozuk yollardan giden "Hayalet Tır"
]

# Potansiyel Sensör Konumları (J Kümesi)
# Senin istediğin o "başta, sonda, ortada" yapısı burada:
konumlar = [
    # BÖLGE 1: ÇIKIŞ (KAZAN)
    "j1_Kazan_Cikis_Ana",  # Ankara çıkışı ana yol (Herkes buradan başlar)
    # BÖLGE 2: KIZILCAHAMAM GEÇİŞİ
    "j2_Kizilcahamam_Merkez",  # Şehir içi geçiş
    "j3_Dag_Yolu_Giris",  # Dağ yoluna sapılan o sapa nokta
    # BÖLGE 3: PINAR İSTASYONU (KRİTİK BÖLGE)
    "j4_Pinar_Istasyonu",  # Mevcut istasyon (Uydu görüntüsündeki yer)
    "j5_Pinar_Arka_Kacak",  # İstasyonun arkasındaki o ince yol
    # BÖLGE 4: GEREDE KAVŞAĞI (BİRLEŞİM)
    "j6_Gerede_Giris_Bogazi",  # D-750'nin bittiği yer
    "j7_Gerede_Koy_Yolu",  # Kavşağı bypass eden köy yolu
    # BÖLGE 5: BOLU YÖNÜ (SONUÇ)
    "j8_D100_Bolu_Duzlugu",  # D-100 Devlet Yolu (Eskipazar sapağı sonrası)
    "j9_O4_Gerede_Giseler",  # Otoyol Gişeleri
    "j10_Eski_Bolu_Yolu",  # Çok eski yol üzeri
]

# Maliyetler (Milyon TL)
# Mantık: Mevcut istasyonlar ucuz, ana yollar orta, dağ/köy yolları (altyapısız) pahalı.
maliyetler = {
    "j1_Kazan_Cikis_Ana": 12,  # Çok geniş yol, büyük TAG lazım
    "j2_Kizilcahamam_Merkez": 9,  # Şehir içi kurulum
    "j3_Dag_Yolu_Giris": 15,  # Dağ başı, elektrik yok (Pahalı)
    "j4_Pinar_Istasyonu": 4,  # Mevcut yer, SADECE SENSÖR (En Ucuz)
    "j5_Pinar_Arka_Kacak": 8,  # Tali yol
    "j6_Gerede_Giris_Bogazi": 11,  # Ana kavşak
    "j7_Gerede_Koy_Yolu": 14,  # Köy yolu, altyapı yok
    "j8_D100_Bolu_Duzlugu": 10,  # Standart kurulum
    "j9_O4_Gerede_Giseler": 6,  # Gişe elektriği var, ucuz
    "j10_Eski_Bolu_Yolu": 16,  # Çok ıssız, çok pahalı
}

# Kapsama Matrisi (a_ij)
kapsama_matrisi = {}
for i in rotalar:
    for j in konumlar:
        kapsama_matrisi[i, j] = 0

# --- SENARYOLARI İŞLEME ---

# i1 (Otoyolcu): Kazan'dan çıkar, Pınar'dan geçer, Gerede'den Otoyola girer.
# Yakalanabileceği Yerler: j1, j2, j4 (Pınar), j6 (Gerede), j9 (Otoyol Gişe)
for j in [
    "j1_Kazan_Cikis_Ana",
    "j2_Kizilcahamam_Merkez",
    "j4_Pinar_Istasyonu",
    "j6_Gerede_Giris_Bogazi",
    "j9_O4_Gerede_Giseler",
]:
    kapsama_matrisi["i1_Full_Durust_Otoyolcu", j] = 1

# i2 (Devlet Yolcu): Kazan -> Pınar -> Gerede -> D100.
# Yakalanabileceği Yerler: j1, j2, j4 (Pınar), j6 (Gerede), j8 (D100)
for j in [
    "j1_Kazan_Cikis_Ana",
    "j2_Kizilcahamam_Merkez",
    "j4_Pinar_Istasyonu",
    "j6_Gerede_Giris_Bogazi",
    "j8_D100_Bolu_Duzlugu",
]:
    kapsama_matrisi["i2_Durust_DevletYolcu", j] = 1

# i3 (Pınar Kaçağı): Pınar istasyonunun arkasından dolaşır.
# Yakalanabileceği Yerler: j1, j2, j5 (Arka Yol), j6, j8/j9
# NOT: j4'ten (Mevcut İstasyon) kaçar!
for j in [
    "j1_Kazan_Cikis_Ana",
    "j2_Kizilcahamam_Merkez",
    "j5_Pinar_Arka_Kacak",
    "j6_Gerede_Giris_Bogazi",
    "j8_D100_Bolu_Duzlugu",
]:
    kapsama_matrisi["i3_Pinar_Kacagi", j] = 1

# i4 (Dağcı): Kızılcahamam'da dağa sapar, Pınar'ı çok uzaktan geçer.
# Yakalanabileceği Yerler: j1, j3 (Dağ Yolu), j6 (Sonunda Gerede'ye iner), j8/j9
# NOT: j2, j4, j5 hepsini pas geçer.
for j in [
    "j1_Kazan_Cikis_Ana",
    "j3_Dag_Yolu_Giris",
    "j6_Gerede_Giris_Bogazi",
    "j8_D100_Bolu_Duzlugu",
]:
    kapsama_matrisi["i4_Kizilcahamam_Dagcisi", j] = 1

# i5 (Gerede Bypass): Pınar'dan geçer ama Gerede kavşağından köye sapar.
# Yakalanabileceği Yerler: j1, j2, j4 (Pınar), j7 (Köy Yolu), j8
# NOT: j6'dan (Gerede Göbeği) kaçar!
for j in [
    "j1_Kazan_Cikis_Ana",
    "j2_Kizilcahamam_Merkez",
    "j4_Pinar_Istasyonu",
    "j7_Gerede_Koy_Yolu",
    "j8_D100_Bolu_Duzlugu",
]:
    kapsama_matrisi["i5_Gerede_Bypass", j] = 1

# i6 (Hayalet Tır): En eski yolları kullanır.
# Yakalanabileceği Yerler: j1 (Mecbur çıkacak), j10 (Eski yol), j8 (En son D100'e bağlanır)
# NOT: Aradaki her şeyi pas geçer.
for j in ["j1_Kazan_Cikis_Ana", "j10_Eski_Bolu_Yolu", "j8_D100_Bolu_Duzlugu"]:
    kapsama_matrisi["i6_Eski_Bolu_Yolu", j] = 1


# --- 2. MODEL ÇÖZÜMÜ ---
model = pyo.ConcreteModel(name="KGM_Regional_Network")
model.I = pyo.Set(initialize=rotalar)
model.J = pyo.Set(initialize=konumlar)
model.c = pyo.Param(model.J, initialize=maliyetler)
model.a = pyo.Param(model.I, model.J, initialize=kapsama_matrisi)
model.x = pyo.Var(model.J, domain=pyo.Binary)


def obj_rule(m):
    return sum(m.c[j] * m.x[j] for j in m.J)


model.Obj = pyo.Objective(rule=obj_rule, sense=pyo.minimize)


def cover_rule(m, i):
    return sum(m.a[i, j] * m.x[j] for j in m.J) >= 1


model.Constraints = pyo.Constraint(model.I, rule=cover_rule)

# GLPK Yolunu düzeltmeyi unutma!
solver = pyo.SolverFactory("glpk", executable=r"C:\glpk\glpk-4.65\w64\glpsol.exe")
result = solver.solve(model)

# --- SONUÇ EKRANI ---
print("\n" + "=" * 60)
print("   ANKARA-BOLU BÖLGESEL AĞ OPTİMİZASYON SONUÇLARI")
print("=" * 60)

secilenler = [j for j in model.J if pyo.value(model.x[j]) > 0.5]
toplam_maliyet = sum(maliyetler[j] for j in secilenler)

print(f"\n💰 MİNİMUM TOPLAM MALİYET: {toplam_maliyet} Milyon TL")
print(f"📍 KURULACAK İSTASYON SAYISI: {len(secilenler)}\n")

print("--- SEÇİLEN STRATEJİK NOKTALAR ---")
for j in secilenler:
    print(f" >> {j} (Maliyet: {maliyetler[j]} M)")

print("\n" + "-" * 60)
print("ROTALAR NASIL YAKALANDI?")
print("-" * 60)
for i in rotalar:
    yakalayanlar = [j for j in secilenler if kapsama_matrisi[i, j] == 1]
    print(f"🚚 {i:<25} -> Yakalandığı Yer: {yakalayanlar}")


   ANKARA-BOLU BÖLGESEL AĞ OPTİMİZASYON SONUÇLARI

💰 MİNİMUM TOPLAM MALİYET: 12 Milyon TL
📍 KURULACAK İSTASYON SAYISI: 1

--- SEÇİLEN STRATEJİK NOKTALAR ---
 >> j1_Kazan_Cikis_Ana (Maliyet: 12 M)

------------------------------------------------------------
ROTALAR NASIL YAKALANDI?
------------------------------------------------------------
🚚 i1_Full_Durust_Otoyolcu   -> Yakalandığı Yer: ['j1_Kazan_Cikis_Ana']
🚚 i2_Durust_DevletYolcu     -> Yakalandığı Yer: ['j1_Kazan_Cikis_Ana']
🚚 i3_Pinar_Kacagi           -> Yakalandığı Yer: ['j1_Kazan_Cikis_Ana']
🚚 i4_Kizilcahamam_Dagcisi   -> Yakalandığı Yer: ['j1_Kazan_Cikis_Ana']
🚚 i5_Gerede_Bypass          -> Yakalandığı Yer: ['j1_Kazan_Cikis_Ana']
🚚 i6_Eski_Bolu_Yolu         -> Yakalandığı Yer: ['j1_Kazan_Cikis_Ana']


In [ ]:
import pyomo.environ as pyo

rotalar = [
    "i1_Full_Durust_Otoyolcu",
    "i2_Durust_DevletYolcu",
    "i3_Pinar_Kacagi",
    "i4_Kizilcahamam_Dagcisi",
    "i5_Gerede_Bypass",
    "i6_Eski_Bolu_Yolu",
    "i7_Camlidere_Katilimi",
]

konumlar = [
    "j1_Kazan_Cikis_Ana",
    "j2_Kizilcahamam_Merkez",
    "j3_Dag_Yolu_Giris",
    "j4_Pinar_Istasyonu",
    "j5_Pinar_Arka_Kacak",
    "j6_Gerede_Giris_Bogazi",
    "j7_Gerede_Koy_Yolu",
    "j8_D100_Bolu_Duzlugu",
    "j9_O4_Gerede_Giseler",
    "j10_Eski_Bolu_Yolu",
]

maliyetler = {
    "j1_Kazan_Cikis_Ana": 45,
    "j2_Kizilcahamam_Merkez": 15,
    "j3_Dag_Yolu_Giris": 20,
    "j4_Pinar_Istasyonu": 4,
    "j5_Pinar_Arka_Kacak": 6,
    "j6_Gerede_Giris_Bogazi": 12,
    "j7_Gerede_Koy_Yolu": 14,
    "j8_D100_Bolu_Duzlugu": 50,
    "j9_O4_Gerede_Giseler": 5,
    "j10_Eski_Bolu_Yolu": 16,
}

kapsama_matrisi = {}
for i in rotalar:
    for j in konumlar:
        kapsama_matrisi[i, j] = 0

for j in [
    "j1_Kazan_Cikis_Ana",
    "j2_Kizilcahamam_Merkez",
    "j4_Pinar_Istasyonu",
    "j6_Gerede_Giris_Bogazi",
    "j9_O4_Gerede_Giseler",
]:
    kapsama_matrisi["i1_Full_Durust_Otoyolcu", j] = 1

for j in [
    "j1_Kazan_Cikis_Ana",
    "j2_Kizilcahamam_Merkez",
    "j4_Pinar_Istasyonu",
    "j6_Gerede_Giris_Bogazi",
    "j8_D100_Bolu_Duzlugu",
]:
    kapsama_matrisi["i2_Durust_DevletYolcu", j] = 1

for j in [
    "j1_Kazan_Cikis_Ana",
    "j2_Kizilcahamam_Merkez",
    "j5_Pinar_Arka_Kacak",
    "j6_Gerede_Giris_Bogazi",
    "j8_D100_Bolu_Duzlugu",
]:
    kapsama_matrisi["i3_Pinar_Kacagi", j] = 1

for j in [
    "j1_Kazan_Cikis_Ana",
    "j3_Dag_Yolu_Giris",
    "j6_Gerede_Giris_Bogazi",
    "j8_D100_Bolu_Duzlugu",
]:
    kapsama_matrisi["i4_Kizilcahamam_Dagcisi", j] = 1

for j in [
    "j1_Kazan_Cikis_Ana",
    "j2_Kizilcahamam_Merkez",
    "j4_Pinar_Istasyonu",
    "j7_Gerede_Koy_Yolu",
    "j8_D100_Bolu_Duzlugu",
]:
    kapsama_matrisi["i5_Gerede_Bypass", j] = 1

for j in ["j1_Kazan_Cikis_Ana", "j10_Eski_Bolu_Yolu", "j8_D100_Bolu_Duzlugu"]:
    kapsama_matrisi["i6_Eski_Bolu_Yolu", j] = 1

for j in ["j4_Pinar_Istasyonu", "j6_Gerede_Giris_Bogazi", "j9_O4_Gerede_Giseler"]:
    kapsama_matrisi["i7_Camlidere_Katilimi", j] = 1


model = pyo.ConcreteModel(name="KGM")
model.I = pyo.Set(initialize=rotalar)
model.J = pyo.Set(initialize=konumlar)
model.c = pyo.Param(model.J, initialize=maliyetler)
model.a = pyo.Param(model.I, model.J, initialize=kapsama_matrisi)
model.x = pyo.Var(model.J, domain=pyo.Binary)


def obj_rule(m):
    return sum(m.c[j] * m.x[j] for j in m.J)


model.Obj = pyo.Objective(rule=obj_rule, sense=pyo.minimize)


def cover_rule(m, i):
    return sum(m.a[i, j] * m.x[j] for j in m.J) >= 1


model.Constraints = pyo.Constraint(model.I, rule=cover_rule)

solver = pyo.SolverFactory("glpk", executable=r"C:\glpk\glpk-4.65\w64\glpsol.exe")
result = solver.solve(model)

secilenler = [j for j in model.J if pyo.value(model.x[j]) > 0.5]
toplam_maliyet = sum(maliyetler[j] for j in secilenler)

print(f"\n💰 TOPLAM MALİYET: {toplam_maliyet} Milyon TL")
print(f"📍 SEÇİLEN İSTASYONLAR ({len(secilenler)} Adet):")
for j in secilenler:
    print(f"   ✅ {j} (Maliyet: {maliyetler[j]} M)")


💰 TOPLAM MALİYET: 32 Milyon TL
📍 SEÇİLEN İSTASYONLAR (3 Adet):
   ✅ j4_Pinar_Istasyonu (Maliyet: 4 M)
   ✅ j6_Gerede_Giris_Bogazi (Maliyet: 12 M)
   ✅ j10_Eski_Bolu_Yolu (Maliyet: 16 M)


In [ ]:
import folium

m = folium.Map(location=[40.60, 32.40], zoom_start=9, tiles="CartoDB positron")

# --- 2. KOORDİNATLAR VE MALİYETLER ---
# (Bu koordinatlar yaklaşık gerçek yerlerdir)

# Kritik Noktalar (Sensör Adayları)
locations = {
    "j1_Kazan_Cikis_Ana": {
        "coords": [40.203, 32.685],
        "cost": 45,
        "type": "Pahalı Merkez",
    },
    "j2_Kizilcahamam_Merkez": {"coords": [40.472, 32.652], "cost": 15, "type": "Orta"},
    "j3_Dag_Yolu_Giris": {"coords": [40.485, 32.610], "cost": 20, "type": "Pahalı Dağ"},
    "j4_Pinar_Istasyonu": {
        "coords": [40.558, 32.555],
        "cost": 4,
        "type": "Ucuz (Mevcut)",
    },
    "j5_Pinar_Arka_Kacak": {
        "coords": [40.558, 32.535],
        "cost": 6,
        "type": "Ucuz (Tali)",
    },
    "j6_Gerede_Giris_Bogazi": {
        "coords": [40.785, 32.225],
        "cost": 12,
        "type": "Orta (Stratejik)",
    },
    "j7_Gerede_Koy_Yolu": {"coords": [40.770, 32.180], "cost": 14, "type": "Orta"},
    "j8_D100_Bolu_Duzlugu": {
        "coords": [40.765, 31.950],
        "cost": 50,
        "type": "Çok Pahalı",
    },
    "j9_O4_Gerede_Giseler": {
        "coords": [40.805, 32.205],
        "cost": 5,
        "type": "Ucuz (Gişe)",
    },
    "j10_Eski_Bolu_Yolu": {
        "coords": [40.680, 32.050],
        "cost": 16,
        "type": "Pahalı (Issız)",
    },
}

# Başlangıç Noktaları
start_points = {
    "Ankara Lojistik Üssü": [40.100, 32.580],
    "Çamlıdere Katılımı (Ara Giriş)": [40.510, 32.450],  # i7 rotası için
}

# --- 3. SENSÖR NOKTALARINI EKLE (Maliyete Göre Renkli) ---
fg_sensors = folium.FeatureGroup(name="Potansiyel Sensör Konumları (J)")

for name, data in locations.items():
    # Rengi maliyete göre belirle
    if data["cost"] <= 6:
        color = "green"  # Ucuz, modelin sevdiği
    elif data["cost"] >= 40:
        color = "red"  # Çok pahalı, modelin kaçtığı
    else:
        color = "blue"  # Orta şeker

    fg_sensors.add_child(
        folium.CircleMarker(
            location=data["coords"],
            radius=8 if color != "red" else 12,  # Pahalılar büyük görünsün
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7,
            popup=f"<b>{name}</b><br>Maliyet: {data['cost']} Milyon TL<br>Tip: {data['type']}",
            tooltip=f"{name} ({data['cost']}M TL)",
        )
    )
m.add_child(fg_sensors)

# Başlangıç noktalarını ekle
for name, coords in start_points.items():
    folium.Marker(
        location=coords,
        popup=name,
        icon=folium.Icon(color="black", icon="play", prefix="fa"),
    ).add_to(m)


# --- 4. ROTALARI ÇİZ (Katman Katman) ---

# Rota i1: Tam Dürüst (Otoyolcu) - YEŞİL
# Güzergah: Kazan -> Kızılcahamam -> Pınar -> Gerede -> Otoyol Gişe
route_i1 = [
    start_points["Ankara Lojistik Üssü"],
    locations["j1_Kazan_Cikis_Ana"]["coords"],
    [40.35, 32.66],  # Ara nokta
    locations["j2_Kizilcahamam_Merkez"]["coords"],
    locations["j4_Pinar_Istasyonu"]["coords"],
    [40.65, 32.45],  # Ara nokta
    locations["j6_Gerede_Giris_Bogazi"]["coords"],
    locations["j9_O4_Gerede_Giseler"]["coords"],
    [40.82, 31.90],  # Bolu Otoyol yönü
]
fg_i1 = folium.FeatureGroup(name="i1: Dürüst Otoyolcu (Yeşil)")
fg_i1.add_child(folium.PolyLine(route_i1, color="green", weight=5, opacity=0.7))
m.add_child(fg_i1)


# Rota i3: Pınar Kaçağı (İstasyon Arkası) - TURUNCU
# Güzergah: ... -> Kızılcahamam -> Pınar Arkası (Tali Yol) -> Gerede ...
route_i3 = [
    locations["j2_Kizilcahamam_Merkez"]["coords"],
    [40.52, 32.60],
    locations["j5_Pinar_Arka_Kacak"]["coords"],  # KRİTİK SAPMA
    [40.60, 32.50],
    locations["j6_Gerede_Giris_Bogazi"]["coords"],
]
fg_i3 = folium.FeatureGroup(name="i3: Pınar İstasyon Kaçağı (Turuncu)")
fg_i3.add_child(
    folium.PolyLine(route_i3, color="orange", weight=4, dash_array="10, 10")
)
m.add_child(fg_i3)


# Rota i4: Dağcı (Uzun Bypass) - KIRMIZI
# Güzergah: Kızılcahamam -> Dağ Yolu -> Çerkeş Yolu -> Gerede
route_i4 = [
    locations["j2_Kizilcahamam_Merkez"]["coords"],
    locations["j3_Dag_Yolu_Giris"]["coords"],
    [40.60, 32.75],  # Dağların arkası
    [40.75, 32.50],  # Çerkeş tarafı
    locations["j6_Gerede_Giris_Bogazi"]["coords"],
]
fg_i4 = folium.FeatureGroup(name="i4: Dağ Yolu Kaçağı (Kırmızı)")
fg_i4.add_child(folium.PolyLine(route_i4, color="red", weight=4, opacity=0.6))
m.add_child(fg_i4)


# Rota i7: Sonradan Giren (Ara Katılım) - MOR
# Güzergah: Çamlıdere -> Pınar -> Gerede
route_i7 = [
    start_points["Çamlıdere Katılımı (Ara Giriş)"],
    locations["j4_Pinar_Istasyonu"]["coords"],  # Pınar'a yakalanır!
    locations["j6_Gerede_Giris_Bogazi"]["coords"],
    locations["j9_O4_Gerede_Giseler"]["coords"],
]
fg_i7 = folium.FeatureGroup(name="i7: Ara Katılım (Çamlıdere) - Mor")
fg_i7.add_child(folium.PolyLine(route_i7, color="purple", weight=5, opacity=0.8))
m.add_child(fg_i7)


# --- 5. HARİTAYI KAYDET ---
folium.LayerControl(collapsed=False).add_to(m)
m.save("KGM_Zorlu_Senaryo_Haritasi.html")

print("Harita oluşturuldu: KGM_Zorlu_Senaryo_Haritasi.html")

Harita oluşturuldu: KGM_Zorlu_Senaryo_Haritasi.html
